In [1]:
import pandas as pd

In [2]:
import altair as alt

In [3]:
from altair import expr

In [4]:
from altair import datum

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [6]:
prov_list = ['aceh','bali','banten','bangka belitung','bengkulu','yogyakarta','jakarta','jambi','jawa barat','jawa tengah','jawa timur','kalimantan barat','kalimantan timur','kalimantan tengah','kalimantan selatan','kalimantan utara','kepulauan riau','nusa tenggara barat','sumatera selatan','sumatera barat','sulawesi utara','sumatera utara','sulawesi tenggara','sulawesi selatan','sulawesi tengah','lampung','riau','maluku utara','maluku','papua barat','papua','sulawesi barat','nusa tenggara timur','gorontalo'
            ]

In [7]:
data = pd.read_excel("daily_increment_positif.xlsx")

In [8]:
data = data.set_index("tanggal")

In [9]:
data["total"] = data.sum(1)

In [10]:
prov = data[prov_list]

In [11]:
prov_long = prov.reset_index().melt(id_vars="tanggal")

In [12]:
selection = alt.selection_multi(fields=['variable'])

In [13]:
selection2 = alt.selection_multi(fields=['variable'])

In [14]:
base = alt.Chart(prov_long)

In [15]:
cumsumbase = base.transform_window(
    sort=[{'field': 'tanggal'}],
    frame=[None, 0],
    cumulative_sum='sum(value)',
    groupby = ['variable']
)

In [16]:
cumsumlinear = cumsumbase.mark_line().encode(
    x='tanggal:T',
    y = alt.Y('cumulative_sum:Q',scale=alt.Scale(type="linear"),title = 'kasus'),
    color = alt.Color('variable:N',legend=None)
).transform_filter(selection & selection2).properties(width=400,height=250)

In [17]:
cumsumlog = cumsumbase.mark_line().encode(
    x='tanggal:T',
    y = alt.Y('cum_log:Q',scale=alt.Scale(type="log"),title = 'kasus'),
    color = alt.Color('variable:N',legend=None)
).transform_filter(selection & selection2).transform_calculate(cum_log = expr.if_(datum.cumulative_sum == 0,expr.NaN,datum.cumulative_sum)).properties(width=400,height=250)

In [18]:
cumsumstacked = cumsumbase.mark_area().encode(
    x='tanggal:T',
    y = alt.Y('cumulative_sum:Q',scale=alt.Scale(type="linear"),title = 'kasus',stack="normalize"),
    color = alt.Color('variable:N',legend=None)
).transform_filter(selection & selection2).properties(width=400,height=250)

In [19]:
bar_base = base.mark_bar(size=13).transform_aggregate(prov_sum = 'sum(value)',groupby=['variable']).encode(
    y = alt.Y('variable:N',sort='-x',title = 'provinsi'),
    x = alt.X('prov_sum:Q',title = 'kasus'),
    color = 'variable:N')
bar = bar_base.add_selection(selection).properties(width=300,height=alt.Step(16))

In [20]:
mark = bar_base.mark_text(dx=15 ).encode(text = 'prov_sum:Q',color = alt.value('black')).add_selection(selection2)

In [21]:
charts = (cumsumlinear  & cumsumstacked) |(bar + mark) 

In [22]:
charts.properties(
    title='Indonesia Covid-19'
)

alt.HConcatChart(...)

In [23]:
charts.save('indonesia_covid.json')